# Notebook for trace collection

In [59]:
import chipwhisperer as cw
from tqdm import tnrange, trange
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [69]:
scope = cw.scope()
print(scope)


cwlite Device
sn         = 50203220343043543130343234323033
fw_version = 
    major = 0
    minor = 65
    debug = 0
gain = 
    mode = low
    gain = 0
    db   = 5.5
adc = 
    state          = False
    basic_mode     = low
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 24400
    decimate       = 1
    trig_count     = 14468044
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x1
    adc_phase     = 0
    adc_freq      = 0
    adc_rate      = 0.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 1
    clkgen_freq   = 192000000.0
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_tx
    tio2         = serial_rx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z
    nrst         = high_z
    glitch_h

In [70]:
fw_path = "../firmware/simpleserial-aes-CWLITEARM.hex"
prog = cw.programmers.STM32FProgrammer
scope.default_setup()
cw.program_target(scope, prog, fw_path)
target_type = cw.targets.SimpleSerial
target = cw.target(scope, target_type)

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 400175448                 to 477162263                
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538459                 
scope.clock.adc_rate                     changed from 0.0                       to 29538459.0               
scope.clock.adc_loc

# Collect for NB01

In [50]:
nr_samples = 24000
nr_traces = 10

traces = []
scope.adc.samples = nr_samples

# Decide whether you want to create a key, text pair manually or the ``Basic'' class.
# For manual creation of the key you can use the bytearray() function. Remember to keep the key the same and change the text in the loop.
# For information about `Basic'' class look to https://chipwhisperer.readthedocs.io/en/latest/api.html#basic .

# If you have chosen to use ``Basic'' class, define an instance here.
# [ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]
ktp = cw.ktp.Basic()     
ktp.fixed_key = True

for i in trange(nr_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    traces.append(trace)
    
trace_array = np.asarray([trace.wave for trace in traces])
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])  

filename = "./dataset_nb01.npy"

with open(filename, 'wb') as f:
    np.savez(f, trace=trace_array, textin=textin_array)


## Sanity check
print(trace_array.shape)
print(textin_array)


Capturing traces: 100%|██████████| 10/10 [00:01<00:00,  8.29it/s]

(10, 24000)
[[109  12  79  59 127  92  27 179 145 150  90 229 109 170 178  88]
 [214  10  73 171  88  58  92 137 107  83 187   7  86 254 146 134]
 [ 44  61  50  25  28  20  65 151 183  58  39  67  20 246 127 143]
 [ 92 243 213  51 230 217  67 165 240 108 146 252 197 218   3 236]
 [ 84  74  73 203 228  15  52 191 135 234 196  98   9 150 118  81]
 [236 187 203 169 177  46 114  32  35 186 127 117  72  36 211  12]
 [214 158 226 207 142  45 127 114 219 110 242 113  96 133  34 118]
 [ 77 113 140  66  21 250 172  56  90 119  33  78 187 229 120 160]
 [174 192   0 131 201  92  11 245  54  82 252 167 222 188 164  60]
 [108  66 208  11 114 118  74 139 143  97 200 190 110  29  96  54]]


# Collect for NB02


In [51]:
sbox = [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
            0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
            0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
            0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
            0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
            0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
            0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
            0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
            0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
            0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
            0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
            0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
            0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
            0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
            0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
            0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
            0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
            0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
            0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
            0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
            0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
            0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
            0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
            0x54, 0xbb, 0x16]


In [65]:
def output_sbox(plaintext, key): #plaintext and key should be of the same length
    idx = plaintext ^ key
    return [sbox[i] for i in idx]

def get_leakage(textin, keys, byte):
    sbox_values = output_sbox(textin[:,byte], keys[:,byte])
    return sbox_values

In [54]:
nr_samples = 6000
nr_traces = 5000

traces = []
scope.adc.samples = nr_samples

## ASCII for: Pumpkin & ginger
key = [80, 117, 109, 112, 107, 105, 110, 32, 38, 32, 103, 105, 110, 103, 101, 114 ]

ktp = cw.ktp.Basic()     

for i in trange(nr_traces, desc='Capturing traces'):
    _, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    key_bytes = bytearray(key)
    
    trace = cw.capture_trace(scope, target, text, key_bytes)
    if trace is None:
        continue
    traces.append(trace)
    
trace_array = np.asarray([trace.wave for trace in traces])
textin_array = np.asarray([trace.textin for trace in traces])


filename = "./dataset_nb02.npy"

with open(filename, 'wb') as f:
    np.savez(f, trace=trace_array, textin=textin_array)



Capturing traces: 100%|██████████| 5000/5000 [08:04<00:00, 10.32it/s]


# Code for collecting dataset NB03

In [71]:
nr_samples = 5000
nr_traces = 10000

traces = []
scope.adc.samples = nr_samples

## ASCII for: Happy Halloween!
key = [72, 97, 112, 112, 121, 32, 72, 97, 108, 108, 111, 119, 101, 101, 110, 33] 

ktp = cw.ktp.Basic()     

for i in trange(nr_traces, desc='Capturing traces'):
    _, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    key_bytes = bytearray(key)
    
    trace = cw.capture_trace(scope, target, text, key_bytes)
    if trace is None:
        continue
    traces.append(trace)
    
trace_array = np.asarray([trace.wave for trace in traces])
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])  


Capturing traces: 100%|██████████| 10000/10000 [15:54<00:00, 10.47it/s]


In [72]:

filename = "../pretrained-models/dataset_dl.npy"

sensitive_data = np.zeros((nr_traces, 16)) # sbox values for every key+plaintext combo

for i in range(0, 16): ## Calculate for every byte
    values = get_leakage(textin_array, known_keys, i)
    sensitive_data[:,i] = values

POI_begin = 1300 ## Based on experience. This is exactly the window of the first Sbox operation
POI_end = 2000

train_size = 9900
test_size = 100

with open(filename, 'wb') as f:
    np.savez(f, trace=trace_array[:train_size, POI_begin:POI_end], textin=textin_array[:train_size], keys=known_keys[:train_size], sensitive_data=sensitive_data[:train_size])

filename = "./dataset_nb03.npy"

with open(filename, 'wb') as f:
    np.savez(f, trace=trace_array[train_size:, POI_begin:POI_end], textin=textin_array[train_size:])
## Sanity check

print(trace_array.shape)
print(textin_array)
print(known_keys)
print(sensitive_data)


(10000, 5000)
[[102 208 174 ...  71 234  73]
 [177   6 137 ... 187  14  52]
 [182  67 227 ...  98   1 196]
 ...
 [ 76 249  98 ... 116 117 223]
 [142  37  32 ... 157  33  32]
 [254  47 173 ... 178  88  80]]
[[ 72  97 112 ... 101 110  33]
 [ 72  97 112 ... 101 110  33]
 [ 72  97 112 ... 101 110  33]
 ...
 [ 72  97 112 ... 101 110  33]
 [ 72  97 112 ... 101 110  33]
 [ 72  97 112 ... 101 110  33]]
[[ 49. 200.  29. ... 147.  95.  69.]
 [153. 133. 153. ...  29. 208.  89.]
 [187. 147. 220. ... 197. 168. 217.]
 ...
 [242.  70. 201. ... 130. 175. 187.]
 [180.  27.  83. ...  65. 132. 124.]
 [ 78.  47. 193. ...  14.   5. 163.]]
